In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import math

In [3]:
df = pd.read_csv('/content/drive/MyDrive/공모전/주제 1/train_heat.csv')

df

,Unnamed: 0,train_heat.tm,train_heat.branch_id,train_heat.ta,train_heat.wd,train_heat.ws,train_heat.rn_day,train_heat.rn_hr1,train_heat.hm,train_heat.si,train_heat.ta_chi,train_heat.heat_demand
0,1,2021010101,A,-10.1,78.3,0.5,0.0,0.0,68.2,-99.0,-8.2,281
1,2,2021010102,A,-10.2,71.9,0.6,0.0,0.0,69.9,-99.0,-8.6,262
2,3,2021010103,A,-10.0,360.0,0.0,0.0,0.0,69.2,-99.0,-8.8,266
3,4,2021010104,A,-9.3,155.9,0.5,0.0,0.0,65.0,-99.0,-8.9,285
4,5,2021010105,A,-9.0,74.3,1.9,0.0,0.0,63.5,-99.0,-9.2,283
...,...,...,...,...,...,...,...,...,...,...,...,...
499296,499297,2023123119,S,3.2,233.5,0.4,2.5,0.0,91.5,-99.0,2.8,34
499297,499298,2023123120,S,2.9,227.4,0.1,2.5,0.0,92.1,-99.0,2.7,35
499298,499299,2023123121,S,2.1,360.0,0.0,2.5,0.0,93.3,-99.0,1.4,35
499299,499300,2023123122,S,2.2,30.0,1.4,2.5,0.0,95.5,-99.0,1.3,40


In [4]:
df['train_heat.tm'] = pd.to_datetime(df['train_heat.tm'], format='%Y%m%d%H')

In [5]:
df.dtypes

,0
Unnamed: 0,int64
train_heat.tm,datetime64[ns]
train_heat.branch_id,object
train_heat.ta,float64
train_heat.wd,float64
train_heat.ws,float64
train_heat.rn_day,float64
train_heat.rn_hr1,float64
train_heat.hm,float64
train_heat.si,float64


In [6]:
df['train_heat.wd'] = df['train_heat.wd'].replace(-9.9, pd.NA)
df = df.replace(-99, np.nan)
df['train_heat.wd'] = pd.to_numeric(df['train_heat.wd'], errors='coerce')

print(df.isnull().sum())

Unnamed: 0                    0
train_heat.tm                 0
train_heat.branch_id          0
train_heat.ta             12997
train_heat.wd             20404
train_heat.ws             18815
train_heat.rn_day         18626
train_heat.rn_hr1         19154
train_heat.hm             39717
train_heat.si                 0
train_heat.ta_chi            20
train_heat.heat_demand       23
dtype: int64


In [7]:
df = df.interpolate(method='linear', limit_direction = 'both')
print(df.isnull().sum())

<ipython-input-7-480231772>:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df = df.interpolate(method='linear', limit_direction = 'both')


Unnamed: 0                0
train_heat.tm             0
train_heat.branch_id      0
train_heat.ta             0
train_heat.wd             0
train_heat.ws             0
train_heat.rn_day         0
train_heat.rn_hr1         0
train_heat.hm             0
train_heat.si             0
train_heat.ta_chi         0
train_heat.heat_demand    0
dtype: int64


### 지점별 분석 ###

In [17]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

def create_multivariate_sequences(X, y, seq_len=24):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_len):
        X_seq.append(X[i:i+seq_len])
        y_seq.append(y[i+seq_len])
    return np.array(X_seq), np.array(y_seq)

def predict_test_multivariate(test_df, model, scaler_X, scaler_y, feature_cols, seq_len=24):
    test_df = test_df.asfreq('H').fillna(method='ffill')
    X_test_raw = test_df[feature_cols].values
    X_test_scaled = scaler_X.transform(X_test_raw)
    X_test_seq, _ = create_multivariate_sequences(X_test_scaled, np.zeros(len(X_test_scaled)), seq_len)
    y_pred_scaled = model.predict(X_test_seq)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    return y_pred

feature_cols = [
    "train_heat.ta", "train_heat.wd", "train_heat.ws",
    "train_heat.rn_day", "train_heat.rn_hr1",
    "train_heat.hm", "train_heat.si", "train_heat.ta_chi"
]
target_col = "train_heat.heat_demand"

seq_len = 24
results = {}
branch_ids = df['train_heat.branch_id'].unique()

for branch_id in branch_ids:
    print(f"\n▶ 지점 {branch_id} 처리 중...")

    df_branch = df[df['train_heat.branch_id'] == branch_id].copy()
    df_branch = df_branch.set_index('train_heat.tm').asfreq('H').fillna(method='ffill')

    if df_branch.shape[0] < seq_len + 10:
        print(f"지점 {branch_id}의 데이터 부족으로 건너뜁니다.")
        continue

    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()

    X_scaled = scaler_X.fit_transform(df_branch[feature_cols])
    y_scaled = scaler_y.fit_transform(df_branch[[target_col]])

    X, y = create_multivariate_sequences(X_scaled, y_scaled, seq_len=seq_len)

    split_index = int(len(X) * 0.8)
    X_train, X_val = X[:split_index], X[split_index:]
    y_train, y_val = y[:split_index], y[split_index:]

    model = Sequential([
        LSTM(128, return_sequences=True, input_shape=(seq_len, len(feature_cols))),
        Dropout(0.2),
        LSTM(64, return_sequences=False),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    model.compile(loss='mse', optimizer='adam')
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_data=(X_val, y_val),
        callbacks=[early_stop],
        verbose=1
    )

    y_val_pred = model.predict(X_val)
    y_val_true = scaler_y.inverse_transform(y_val)
    y_val_pred_inv = scaler_y.inverse_transform(y_val_pred)

    rmse = np.sqrt(mean_squared_error(y_val_true, y_val_pred_inv))
    r2 = r2_score(y_val_true, y_val_pred_inv)
    print(f"지점 {branch_id} | RMSE: {rmse:.4f} | R²: {r2:.4f}")

    results[branch_id] = {
        "model": model,
        "scaler_X": scaler_X,
        "scaler_y": scaler_y,
        "RMSE": rmse,
        "R2": r2
    }

    ts_index = df_branch.index[seq_len + split_index: seq_len + len(X)]

    plt.figure(figsize=(14, 5))
    plt.plot(ts_index, y_val_true, label='True', linewidth=1)
    plt.plot(ts_index, y_val_pred_inv, label='Predicted', linewidth=1)
    plt.title(f"Branch {branch_id} - Validation Prediction vs True\nRMSE={rmse:.4f}, R²={r2:.4f}")
    plt.xlabel("Time")
    plt.ylabel("Heat Demand")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"branch_{branch_id}_val_plot_multivariate.png")
    plt.close()


▶ 지점 A 처리 중...
Epoch 1/100


<ipython-input-17-542380283>:41: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_branch = df_branch.set_index('train_heat.tm').asfreq('H').fillna(method='ffill')
<ipython-input-17-542380283>:41: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_branch = df_branch.set_index('train_heat.tm').asfreq('H').fillna(method='ffill')
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


657/657 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - loss: 0.0129 - val_loss: 0.0031
Epoch 2/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0034 - val_loss: 0.0017
Epoch 3/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.0027 - val_loss: 0.0016
Epoch 4/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.0025 - val_loss: 0.0017
Epoch 5/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.0025 - val_loss: 0.0016
Epoch 6/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.0024 - val_loss: 0.0017
Epoch 7/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0024 - val_loss: 0.0022
Epoch 8/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.0024 - val_loss: 0.0016
Epoch 9/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.0022 - val_loss: 0.0017
Epoch 10/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.0022 - val_loss: 0.0019
Epoch 11/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 0.0022 - val_loss: 0.0016
Epoch 12/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 9s 8

<ipython-input-17-542380283>:41: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_branch = df_branch.set_index('train_heat.tm').asfreq('H').fillna(method='ffill')
<ipython-input-17-542380283>:41: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_branch = df_branch.set_index('train_heat.tm').asfreq('H').fillna(method='ffill')
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


657/657 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - loss: 0.0117 - val_loss: 0.0020
Epoch 2/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 0.0036 - val_loss: 0.0018
Epoch 3/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 0.0031 - val_loss: 0.0016
Epoch 4/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.0029 - val_loss: 0.0015
Epoch 5/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.0030 - val_loss: 0.0019
Epoch 6/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.0025 - val_loss: 0.0017
Epoch 7/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.0024 - val_loss: 0.0013
Epoch 8/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.0025 - val_loss: 0.0016
Epoch 9/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 0.0023 - val_loss: 0.0015
Epoch 10/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 0.0023 - val_loss: 0.0019
Epoch 11/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.0022 - val_loss: 0.0017
Epoch 12/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 9s 8m

...

657/657 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - loss: 0.0076 - val_loss: 0.0024
Epoch 2/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 0.0029 - val_loss: 0.0026
Epoch 3/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0027 - val_loss: 0.0018
Epoch 4/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0026 - val_loss: 0.0019
Epoch 5/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.0024 - val_loss: 0.0016
Epoch 6/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 0.0022 - val_loss: 0.0017
Epoch 7/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.0023 - val_loss: 0.0014
Epoch 8/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.0023 - val_loss: 0.0017
Epoch 9/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 0.0021 - val_loss: 0.0016
Epoch 10/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 0.0022 - val_loss: 0.0015
Epoch 11/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.0020 - val_loss: 0.0017
Epoch 12/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/

<ipython-input-17-542380283>:41: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_branch = df_branch.set_index('train_heat.tm').asfreq('H').fillna(method='ffill')
<ipython-input-17-542380283>:41: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_branch = df_branch.set_index('train_heat.tm').asfreq('H').fillna(method='ffill')
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - loss: 0.0170 - val_loss: 0.0061
Epoch 2/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.0069 - val_loss: 0.0065
Epoch 3/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.0061 - val_loss: 0.0059
Epoch 4/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.0060 - val_loss: 0.0055
Epoch 5/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.0059 - val_loss: 0.0059
Epoch 6/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.0056 - val_loss: 0.0057
Epoch 7/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.0057 - val_loss: 0.0049
Epoch 8/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0054 - val_loss: 0.0058
Epoch 9/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.0054 - val_loss: 0.0067
Epoch 10/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.0053 - val_loss: 0.0058
Epoch 11/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0051 - val_loss: 0.0066
Epoch 12/100
657/657 ━━━━━━━━━━━━━━━━━━━━

<ipython-input-17-542380283>:41: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_branch = df_branch.set_index('train_heat.tm').asfreq('H').fillna(method='ffill')
<ipython-input-17-542380283>:41: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_branch = df_branch.set_index('train_heat.tm').asfreq('H').fillna(method='ffill')
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - loss: 0.0046 - val_loss: 0.0051
Epoch 2/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 0.0028 - val_loss: 0.0054
Epoch 3/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.0025 - val_loss: 0.0045
Epoch 4/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.0024 - val_loss: 0.0046
Epoch 5/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0022 - val_loss: 0.0055
Epoch 6/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0021 - val_loss: 0.0058
Epoch 7/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.0021 - val_loss: 0.0053
Epoch 8/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.0020 - val_loss: 0.0049
Epoch 9/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.0019 - val_loss: 0.0060
Epoch 10/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0019 - val_loss: 0.0046
Epoch 11/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.0018 - val_loss: 0.0049
Epoch 12/100
657/657 ━━━━━━━━━━━━━━━━━━━

### Test Result ###

In [18]:
test_df = pd.read_csv('/content/drive/MyDrive/공모전/주제 1/test_heat.csv')

test_df

,TM,branch_ID,TA,WD,WS,RN_DAY,RN_HR1,HM,SI,ta_chi,heat_demand
0,2023123100,A,1.9,107.7,2.3,14.5,0.0,94.9,-99.0,-0.5,NaN
1,2023123101,A,2.1,82.6,2.4,0.0,0.0,95.2,-99.0,0.1,NaN
2,2023123102,A,2.4,71.5,2.6,0.0,0.0,95.5,-99.0,0.5,NaN
3,2023123103,A,2.3,89.3,2.7,0.0,0.0,95.9,-99.0,0.3,NaN
4,2023123104,A,2.2,99.1,2.4,0.0,0.0,96.2,-99.0,0.4,NaN
...,...,...,...,...,...,...,...,...,...,...,...
167366,2024123120,S,-1.1,360.0,0.0,0.0,0.0,45.8,-99.0,-1.7,NaN
167367,2024123121,S,-1.3,360.0,0.0,0.0,0.0,48.3,-99.0,-2.3,NaN
167368,2024123122,S,-2.4,360.0,0.0,0.0,0.0,60.0,-99.0,-3.1,NaN
167369,2024123123,S,-3.6,360.0,0.0,0.0,0.0,65.7,-99.0,-3.9,NaN


In [19]:
test_df = test_df.rename(columns={
    "TM": "train_heat.tm",
    "branch_ID": "train_heat.branch_id",
    "TA": "train_heat.ta",
    "WD": "train_heat.wd",
    "WS": "train_heat.ws",
    "RN_DAY": "train_heat.rn_day",
    "RN_HR1": "train_heat.rn_hr1",
    "HM": "train_heat.hm",
    "SI": "train_heat.si",
    "ta_chi": "train_heat.ta_chi"
})

test_df['train_heat.tm'] = pd.to_datetime(test_df['train_heat.tm'], format='%Y%m%d%H')
test_df['train_heat.si'] = test_df['train_heat.si'].replace(-99, 0)
test_df['train_heat.wd'] = test_df['train_heat.wd'].replace(-9.9, pd.NA)
test_df = test_df.replace(-99, np.nan)
test_df['train_heat.wd'] = pd.to_numeric(test_df['train_heat.wd'], errors='coerce')
test_df = test_df.interpolate(method='linear', limit_direction = 'both')
print(test_df.isnull().sum())

train_heat.tm                0
train_heat.branch_id         0
train_heat.ta                0
train_heat.wd                0
train_heat.ws                0
train_heat.rn_day            0
train_heat.rn_hr1            0
train_heat.hm                0
train_heat.si                0
train_heat.ta_chi            0
heat_demand             167371
dtype: int64


<ipython-input-19-3745594510>:19: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  test_df = test_df.interpolate(method='linear', limit_direction = 'both')


In [20]:
feature_cols = [
    "train_heat.ta", "train_heat.wd", "train_heat.ws",
    "train_heat.rn_day", "train_heat.rn_hr1",
    "train_heat.hm", "train_heat.si", "train_heat.ta_chi"
]

# 시퀀스 생성 함수
def create_multivariate_sequences(X, seq_len=24):
    X_seq = []
    for i in range(len(X) - seq_len):
        X_seq.append(X[i:i+seq_len])
    return np.array(X_seq)

# 예측 함수 (보간 제거 및 역변환 안정성 추가)
def predict_test_multivariate(test_df, model, scaler_X, scaler_y, feature_cols, seq_len=24):
    X_raw = test_df[feature_cols].values
    X_scaled = scaler_X.transform(X_raw)
    X_seq = create_multivariate_sequences(X_scaled, seq_len)

    y_pred_scaled = model.predict(X_seq)
    if len(y_pred_scaled.shape) == 1:
        y_pred_scaled = y_pred_scaled.reshape(-1, 1)

    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    return y_pred

# 예측 저장
submission_rows = []
seq_len = 24

# 지점별 예측 수행
for branch_id in test_df["train_heat.branch_id"].unique():
    test_branch = test_df[test_df["train_heat.branch_id"] == branch_id].copy()
    test_branch = test_branch.set_index("train_heat.tm")
    test_branch = test_branch.sort_index()

    if branch_id not in results:
        print(f"⚠️ 학습 모델이 없는 지점 {branch_id}는 건너뜁니다.")
        continue

    model = results[branch_id]["model"]
    scaler_X = results[branch_id]["scaler_X"]
    scaler_y = results[branch_id]["scaler_y"]

    y_pred = predict_test_multivariate(test_branch, model, scaler_X, scaler_y, feature_cols, seq_len=seq_len)
    pred_start_index = test_branch.index[seq_len:]

    for tm, pred in zip(pred_start_index, y_pred):
        submission_rows.append({
            "train_heat.tm": tm.strftime("%Y-%m-%d %H:%M:%S"),
            "train_heat.branch_id": branch_id,
            "predicted_heat_demand": round(float(pred[0]), 1)
        })

# 예측 결과 DataFrame 생성
submission_df = pd.DataFrame(submission_rows)

# 예측 범위 필터링 (선택사항)
submission_df["train_heat.tm"] = pd.to_datetime(submission_df["train_heat.tm"])
submission_df = submission_df[
    (submission_df["train_heat.tm"] >= "2024-01-01") &
    (submission_df["train_heat.tm"] <= "2025-01-01")
]

# 저장
submission_df = submission_df.sort_values(by=["train_heat.branch_id", "train_heat.tm"])
submission_df.to_csv("predicted_heat_demand_submission.csv", index=False)
print("✅ 예측 완료! 'predicted_heat_demand_submission.csv' 저장됨.")

# 결과 병합 및 저장
test_df["train_heat.tm"] = pd.to_datetime(test_df["train_heat.tm"])
merged_df = pd.merge(
    test_df,
    submission_df,
    on=["train_heat.tm", "train_heat.branch_id"],
    how="left"
)
merged_df["train_heat.tm_formatted"] = merged_df["train_heat.tm"].dt.strftime("%Y%m%d%H")
merged_df.to_csv("predicted_heat_demand_with_features.csv", index=False)

 32/275 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


275/275 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
 48/275 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


275/275 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
 49/275 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

...

275/275 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
 42/275 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


275/275 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
 47/275 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


275/275 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
✅ 예측 완료! 'predicted_heat_demand_submission.csv' 저장됨.
